# 🐦 Twitter Sentiment Analysis in Spanish Tweets  

## 👥 Authors
- **Kristopher Javier Alvarado López** — Carné: 21188  
- **Emilio Jose Solano Orozco** — Carné: 21212  

---

## 📖 Project Description
Este proyecto consiste en el desarrollo de un **pipeline completo de Procesamiento de Lenguaje Natural (NLP)** aplicado a tweets en español.  
El objetivo es analizar el sentimiento expresado en los mensajes (positivo, negativo o neutro) integrando las siguientes etapas:  

1. **Preprocesamiento del texto**: limpieza, tokenización, lematización, normalización y corrección ortográfica.  
2. **Representación de texto**: Bag of Words (BoW), TF-IDF, matrices de co-ocurrencia y embeddings.  
3. **Modelos probabilísticos**: construcción de modelos N-gram y evaluación con entropía y perplejidad.  
4. **Modelos de clasificación**: aplicación de algoritmos supervisados para la detección automática de sentimiento.  
5. **Evaluación y discusión crítica** de resultados.  

---

## 🌍 English Abstract
This project presents the development of a **complete NLP pipeline** for sentiment analysis in Spanish tweets.  
The main goal is to evaluate different text preprocessing techniques, representation models, and machine learning approaches to classify tweets into sentiment categories (positive, negative, neutral). The project combines classical probabilistic language models with modern embedding-based techniques, offering a critical analysis of their performance on real-world social media data.


In [2]:
import pandas as pd

# Cargar dataset
df = pd.read_csv("data/sentiment_analysis_dataset.csv")

# Vista rápida
print("Columnas originales:", df.columns.tolist())
print("Tamaño del dataset:", df.shape)
df.head()


Columnas originales: ['user', 'text', 'date', 'emotion', 'sentiment']
Tamaño del dataset: (2590, 5)


,user,text,date,emotion,sentiment
0,@erreborda,termine bien abrumado después de hoy,"Jan 6, 2024 · 2:53 AM UTC",overwhelmed,scared
1,@shpiderduck,me siento abrumado,"Jan 6, 2024 · 2:35 AM UTC",overwhelmed,scared
2,@Alex_R_art,Me siento un poco abrumado por la cantidad de ...,"Jan 6, 2024 · 12:20 AM UTC",overwhelmed,scared
3,@anggelinaa97,Salvador la única persona que no la ha abrumad...,"Jan 5, 2024 · 10:38 PM UTC",overwhelmed,scared
4,@diegoreyesvqz,Denme un helado o algo que ando full abrumado.,"Jan 5, 2024 · 8:38 PM UTC",overwhelmed,scared


In [3]:
# Eliminar la columna 'sentiment'
df = df.drop(columns=["sentiment"])

# Revisar cambios
print("Columnas después de eliminar 'sentiment':", df.columns.tolist())
df.head()


Columnas después de eliminar 'sentiment': ['user', 'text', 'date', 'emotion']


,user,text,date,emotion
0,@erreborda,termine bien abrumado después de hoy,"Jan 6, 2024 · 2:53 AM UTC",overwhelmed
1,@shpiderduck,me siento abrumado,"Jan 6, 2024 · 2:35 AM UTC",overwhelmed
2,@Alex_R_art,Me siento un poco abrumado por la cantidad de ...,"Jan 6, 2024 · 12:20 AM UTC",overwhelmed
3,@anggelinaa97,Salvador la única persona que no la ha abrumad...,"Jan 5, 2024 · 10:38 PM UTC",overwhelmed
4,@diegoreyesvqz,Denme un helado o algo que ando full abrumado.,"Jan 5, 2024 · 8:38 PM UTC",overwhelmed


# 🛠️ 1. Preprocesamiento del Corpus  

En esta sección realizamos el **preprocesamiento de los tweets en español** para preparar el corpus antes de aplicar modelos de NLP.  

Las tareas principales son:  

1. **Limpieza del texto**: conversión a minúsculas, eliminación de puntuación, URLs, menciones y stopwords.  
2. **Tokenización**: separación de cada oración en palabras.  
3. **Lematización y stemming**: reducción de palabras a su raíz o forma base.  
4. **Uso del algoritmo de Levenshtein**: detección de similitudes y posibles errores ortográficos en el texto.  

---

In [6]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Descargar stopwords (solo primera vez)
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

# 🔹 Función de limpieza
def clean_text(text):
    text = text.lower()  # minúsculas
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # URLs
    text = re.sub(r"@\w+", "", text)  # menciones
    text = re.sub(r"#\w+", "", text)  # hashtags
    text = re.sub(r"[^\w\s]", "", text)  # puntuación
    text = " ".join([word for word in text.split() if word not in stop_words])  # stopwords
    return text.strip()

# 🔹 Aplicar limpieza
df["clean_text"] = df["text"].astype(str).apply(clean_text)
df[["text", "clean_text"]].head(10)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alvar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,text,clean_text
0,termine bien abrumado después de hoy,termine bien abrumado después hoy
1,me siento abrumado,siento abrumado
2,Me siento un poco abrumado por la cantidad de ...,siento abrumado cantidad cosas quiero dibujar ...
3,Salvador la única persona que no la ha abrumad...,salvador única persona abrumado versiones
4,Denme un helado o algo que ando full abrumado.,denme helado ando full abrumado
5,"Estoy abrumado de airdrops , de youtube y de t...",abrumado airdrops youtube imposible gestionarl...
6,"#MicroCuento: A veces, sin motivo aparente, o,...",veces motivo aparente bien razones quizás fúti...
7,"Oh, las vacaciones. Tesoros inciertos, venider...",oh vacaciones tesoros inciertos venideros prec...
8,me siento muy abrumado,siento abrumado
9,Consejo que nadie pidió: Si un día te siente...,consejo nadie pidió si día sientes abrumado tr...


In [7]:
import stanza

# Descargar modelo de español (solo la primera vez)
stanza.download("es")

# Crear pipeline en español
nlp = stanza.Pipeline("es", processors="tokenize,lemma", use_gpu=False)

# Función para tokenizar y lematizar
def tokenize_lemmatize(text):
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

# Aplicar al corpus
df["lemmas"] = df["clean_text"].apply(tokenize_lemmatize)
df[["clean_text", "lemmas"]].head(10)


c:\Users\Alvar\OneDrive\Documentos\SemestreX\Natural Language Process\spanish-twitter-sentiment-nlp\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-28 21:06:07 INFO: Downloaded file to C:\Users\Alvar\stanza_resources\resources.json
2025-09-28 21:06:07 INFO: Downloading default packages for language: es (Spanish) ...
2025-09-28 21:07:36 INFO: Downloaded file to C:\Users\Alvar\stanza_resources\es\default.zip
2025-09-28 21:07:43 INFO: Finished downloading models and saved to C:\Users\Alvar\stanza_resources
2025-09-28 21:07:43 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-09-28 21:07:44 INFO: Downloaded file to C:\Users\Alvar\stanza_resources\resources.j

,clean_text,lemmas
0,termine bien abrumado después hoy,"[terminar, bien, abrumado, después, hoy]"
1,siento abrumado,"[sentir, abrumado]"
2,siento abrumado cantidad cosas quiero dibujar ...,"[sentir, abrumado, cantidad, cosa, querer, dib..."
3,salvador única persona abrumado versiones,"[salvador, único, persona, abrumado, versión]"
4,denme helado ando full abrumado,"[denme, helado, ando, full, abrumado]"
5,abrumado airdrops youtube imposible gestionarl...,"[abrumado, airdrop, youtube, imposible, gestio..."
6,veces motivo aparente bien razones quizás fúti...,"[vez, motivo, aparente, bien, razón, quizás, f..."
7,oh vacaciones tesoros inciertos venideros prec...,"[oh, vacaciones, tesoro, incierto, venidero, p..."
8,siento abrumado,"[sentir, abrumado]"
9,consejo nadie pidió si día sientes abrumado tr...,"[consejo, nadie, pedir, si, día, sentir, abrum..."
